In [1]:
# Libraries
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
import pygame
import sys
from arrow import draw_arrow

pygame 2.5.2 (SDL 2.28.3, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Constants (in inches)
SCALE = 7               # Scale of other constants
W = [60, 44]            # Table dimensions
H = [104, 88]
POCKET_RADIUS = 2.25
BALL_MASS = 210         # Ball specs
BALL_RADIUS = 1.125
MAX_SPEED = 2           # Max speed of ball
FRIC = 0.99             # Friction

In [3]:
MAX_REBOUND = 3         # Maximum number of rebounds
RADIUS_SUM = POCKET_RADIUS + BALL_RADIUS
pos_range = np.array([[RADIUS_SUM, W[1] - RADIUS_SUM], [RADIUS_SUM, H[1] - RADIUS_SUM]]) * SCALE # Range of positions

# Random initial conditions
def rand_init():
    dir = rnd.randint(0, 360)
    cue_pos = rnd.randint(np.round(pos_range[0]), np.round(pos_range[1]), size=2)
    ball_pos = rnd.randint(np.round(pos_range[0]), np.round(pos_range[1]), size=2)
    return dir, cue_pos, ball_pos

# Initial conditions with cue ball close to the 8-ball
def close_init(radius=10*SCALE): # Change default parameters later
    center = rnd.uniform([pos_range[0,0] + radius, pos_range[0,1] - radius], [pos_range[1,0] + radius, pos_range[1,1] - radius], size=2)
    angles = rnd.rand(2) * 2 * np.pi
    cue_pos = np.round(center + radius * np.array([np.cos(angles[0]), np.sin(angles[0])]))
    ball_pos = np.round(center + radius * np.array([np.cos(angles[1]), np.sin(angles[1])]))
    dir = np.round(np.arctan2(ball_pos[1] - cue_pos[1], ball_pos[0] - cue_pos[0]) * 180 / np.pi)
    return dir, cue_pos, ball_pos

In [4]:
# Initialize Pygame
pygame.init()

# Constants
PADDING = SCALE * (W[0]//15)
WIDTH = SCALE * W[0] + PADDING
HEIGHT = SCALE * H[0] + PADDING
X_OFFSET = (WIDTH - SCALE * W[1])//2
Y_OFFSET = (HEIGHT - SCALE * H[1])//2

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BROWN = (139, 69, 19)
RED = (128, 0, 0)
GREEN = (0, 128, 0)
CUE_COLOR = WHITE
BALL_COLOR = BLACK

# Create background
pygame.display.set_caption("Billiards Simulation")
screen = pygame.display.set_mode((WIDTH, HEIGHT))
screen.fill(WHITE)
pockets = np.zeros((6, 2))
for i in range(6):
    x = (i%2) * SCALE * W[1] + X_OFFSET
    y = (i//2) * SCALE * H[1]//2 + Y_OFFSET
    pockets[i] = np.array([x, y])

# Start simulation
file = open("pocketed/speed=2(4).txt", 'a')
running, init, play = True, True, False
while running:
    if init:
        dir, cue_pos, ball_pos = close_init(SCALE * 5)
        init_cue_pos, init_ball_pos = cue_pos.copy(), ball_pos.copy()
        print(f"Initial conditions: dir = {dir}, cue_pos = {cue_pos}, ball_pos = {ball_pos}")
        cue_vel = np.zeros(2)
        ball_vel = np.zeros(2)
        CUE_COLOR = WHITE
        BALL_COLOR = BLACK
        init = False
        play = False
        
    # keys = pygame.key.get_pressed()
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                init = True
            if event.key == pygame.K_RETURN:
                cue_vel = SCALE * MAX_SPEED * np.array([np.cos(dir * np.pi / 180), np.sin(dir * np.pi / 180)])
                play = True
        if event.type == pygame.QUIT:
            running = False
        
    # Clear screen
    pygame.draw.rect(screen, BROWN, ((WIDTH - SCALE * W[0])//2, (HEIGHT - SCALE * H[0])//2, SCALE * W[0], SCALE * H[0]), border_radius=SCALE * W[0] // 10)
    pygame.draw.rect(screen, GREEN, (X_OFFSET, Y_OFFSET, SCALE * W[1], SCALE * H[1]))
    for p in pockets:
        pygame.draw.circle(screen, BLACK, (p[0], p[1]), SCALE * POCKET_RADIUS)
    
    # Draw arrow
    if not play:
        start = (np.round(cue_pos[0]) + X_OFFSET, np.round(cue_pos[1]) + Y_OFFSET)
        end = np.add(start, SCALE * 10 * np.array([np.cos(dir * np.pi / 180), np.sin(dir * np.pi / 180)]))
        end = tuple(end)
        draw_arrow(screen, pygame.math.Vector2(start), pygame.math.Vector2(end), BLACK, 2, 10, 6)
    
    # Draw balls
    cue = pygame.draw.circle(screen, CUE_COLOR, (np.round(cue_pos[0]) + X_OFFSET, np.round(cue_pos[1]) + Y_OFFSET), SCALE * BALL_RADIUS)
    ball = pygame.draw.circle(screen, BALL_COLOR, (np.round(ball_pos[0]) + X_OFFSET, np.round(ball_pos[1]) + Y_OFFSET), SCALE * BALL_RADIUS)
    
    # Update the screen
    pygame.display.flip()
    
    # Boundary checking
    for pos, vel in zip([cue_pos, ball_pos], [cue_vel, ball_vel]):
        # Check for pocketed balls
        d = np.linalg.norm((pockets - (pos + np.array([X_OFFSET, Y_OFFSET]))) / SCALE, axis=1)
        if (vel > 0).any() and d.min() < POCKET_RADIUS:
            vel *= 0
            if pos is cue_pos:
                CUE_COLOR = RED
            else:
                # file.write(f"dir = {dir}, cue_pos = {init_cue_pos}, ball_pos = {init_ball_pos}, pocket = {d.argmin() + 1}\n")
                print(f"\tBall pocketed at pocket {d.argmin() + 1}")
                BALL_COLOR = RED
                
        # in_x_pockets = pos[0] - SCALE * BALL_RADIUS <= SCALE * POCKET_RADIUS or pos[0] + SCALE * BALL_RADIUS >= SCALE * (W[1] - POCKET_RADIUS)
        # in_y_pockets = pos[1] - SCALE * BALL_RADIUS <= SCALE * POCKET_RADIUS or (pos[1] - SCALE * BALL_RADIUS >= SCALE * (H[1] / 2 - POCKET_RADIUS) and pos[1] + SCALE * BALL_RADIUS <= SCALE *( H[1] / 2 + POCKET_RADIUS)) or pos[1] + SCALE * BALL_RADIUS >= SCALE * (H[1] - POCKET_RADIUS)
        if pos[0] - SCALE * BALL_RADIUS <= 0:
            pos[0] = SCALE * BALL_RADIUS
            vel[0] *= -1
        if pos[0] + SCALE * BALL_RADIUS >= SCALE * W[1]:
            pos[0] = SCALE * W[1] - SCALE * BALL_RADIUS
            vel[0] *= -1
        if pos[1] - SCALE * BALL_RADIUS <= 0:
            pos[1] = SCALE * BALL_RADIUS
            vel[1] *= -1
        if pos[1] + SCALE * BALL_RADIUS >= SCALE * H[1]:
            pos[1] = SCALE * H[1] - SCALE * BALL_RADIUS
            vel[1] *= -1
            
    # Elastic collision between balls
    if np.linalg.norm(cue_pos - ball_pos) < 2 * SCALE * BALL_RADIUS:
        diff_pos = cue_pos - ball_pos
        change = np.dot(cue_vel - ball_vel, diff_pos) * diff_pos / np.dot(diff_pos, diff_pos)
        cue_vel -= change
        ball_vel += change
    
    # Apply friction to slow down the ball
    cue_vel *= FRIC
    ball_vel *= FRIC
    
    # Update ball positions
    cue_pos += cue_vel
    ball_pos += ball_vel
    
pygame.quit()
file.close()

Initial conditions: dir = -137.0, cue_pos = [ 89. 253.], ball_pos = [ 74. 239.]
Initial conditions: dir = -62.0, cue_pos = [ 63. 398.], ball_pos = [ 90. 348.]
Initial conditions: dir = 108.0, cue_pos = [ 83. 443.], ball_pos = [ 63. 503.]
Initial conditions: dir = -51.0, cue_pos = [ 26. 496.], ball_pos = [ 39. 480.]
Initial conditions: dir = 65.0, cue_pos = [ 67. 466.], ball_pos = [ 90. 515.]
Initial conditions: dir = -63.0, cue_pos = [ 36. 294.], ball_pos = [ 37. 292.]
Initial conditions: dir = 137.0, cue_pos = [ 42. 269.], ball_pos = [ 29. 281.]
	Ball pocketed at pocket 3
Initial conditions: dir = 90.0, cue_pos = [ 24. 402.], ball_pos = [ 24. 404.]
Initial conditions: dir = 177.0, cue_pos = [ 91. 325.], ball_pos = [ 25. 328.]
Initial conditions: dir = 175.0, cue_pos = [ 83. 376.], ball_pos = [ 30. 381.]
	Ball pocketed at pocket 6
Initial conditions: dir = -77.0, cue_pos = [ 24. 285.], ball_pos = [ 30. 260.]
Initial conditions: dir = -96.0, cue_pos = [ 53. 577.], ball_pos = [ 46. 510.]

: 